### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:

#As we calculate each part of our District Summary dataframe, we will be appending
#each value to a dataframe list to save ourselves from typing in each unique value
#into the dataframe at the end.
final_df_list = []


#Number of Schools (Note: Each one of these shows how I calculated the desired query and
#appended it to our final dataframe list.)
num_schools = len(school_data_complete["school_name"].unique())
final_df_list.append(num_schools)

#Number of Students
num_students = school_data_complete["student_name"].count()
final_df_list.append(num_students)

#Budget. To do this, I iterated over the series of unique budgets and added
#each budget to a counter. There may be an easier way to do this, but this
#is what I came up with.
budget_unique_series = school_data_complete["budget"].unique()

total_budget = 0
for budget in budget_unique_series:
    total_budget = total_budget + budget


final_df_list.append(total_budget)

#Average Math Score
avg_math_score = school_data_complete['math_score'].mean()
final_df_list.append(avg_math_score)

#Average Reading Score
avg_reading_score = school_data_complete['reading_score'].mean()
final_df_list.append(avg_reading_score)

#Reading Passing Rate
math_passing = school_data_complete.loc[school_data_complete["math_score"] >= 70, :]
math_passing = math_passing["Student ID"].count()/num_students
final_df_list.append(math_passing*100)


#Reading Passing Rate
reading_passing = school_data_complete.loc[school_data_complete["reading_score"] >= 70, :]
reading_passing = reading_passing["Student ID"].count()/num_students
final_df_list.append(reading_passing*100)

# print(math_passing)
#Overall Passing Rate (Didn't quite get the same answer...)
overall_passing = (math_passing.max() + reading_passing.max())/2

# overall_passing = overall_passing["Student ID"].count()/num_students
final_df_list.append(overall_passing*100)



#Puts All of My Columns into a list
columns = [
    "Total Schools", "Total Students", "Total Budget",
    "Average Math Score", "Average Reading Score", "Passing Math", "Passing Reading",
     "Overall Passing Rate"
]


#Loops the columns and final_df lists to put them into a dictionary that is easy to turn into
#a data frame.
df_dict = {}
column_index = 0
for value in final_df_list:
    df_dict[columns[column_index]] = [value]
    column_index = column_index + 1
    
#Turns df_dict into a DataFrame
Summary_df = pd.DataFrame(df_dict)

#Maps all of our columns to a better format.
Summary_df["Average Math Score"] = Summary_df["Average Math Score"].map("{:.2f}".format)
Summary_df["Average Reading Score"] = Summary_df["Average Reading Score"].map("{:.2f}".format)
Summary_df["Passing Math"] = Summary_df["Passing Math"].map("{:.2f}%".format)
Summary_df["Passing Reading"] = Summary_df["Passing Reading"].map("{:.2f}%".format)
Summary_df["Total Budget"] = Summary_df["Total Budget"].map("${:,.2f}".format)
Summary_df["Total Students"] = Summary_df["Total Students"].map("{:,}".format)
Summary_df["Overall Passing Rate"] = Summary_df["Overall Passing Rate"].map("{:.2f}%".format)
Summary_df
        

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,80.39%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [3]:
#This technique is a little weird, dictionary-heavy,
#and probably inefficient. Originally, I started with
#this technique, but ended up at a dead end and then tried a more efficient technique
#that used the 'groupby' function. However, I failed to save the file correctly before
#doing a git pull, and thus, lost my progress. The only progress that showed up was
#my half-complete, dictionary-heavy first attempt. Rather than start from scratch,
#I figured out how to get this to work.

#This may sound convoluted, but the goal is to create a list of dictionaries of dictionaries.
#The overall list is a list of dictionaries for each school, and within each dictionary
#is a dictionary mapping each of a school's properties (ex. buget) to its value (ex. $5k)
#From there, we will make a dataframe of each school and then merge them together. This doesn't
#use the intended Groupby function, which likely would have made this exercise easier.
#I do not recommend this technique.

#Creates a blank list that we will add to with each loop.
total_school_dict_list = []

#Loops over every unique school
for school in school_data_complete["school_name"].unique():
    
    #Creates a blank dictionary for each school
    school_dict = {}
    
    #Saves school_name before I change the school variable.
    school_name = school
    
    #Uses '.loc' to find the data for that particular school
    school = school_data_complete.loc[(school_data_complete["school_name"] == school),:]

    #Takes the school type and adds it to the dictionary
    school_dict['School Type'] = school['type'].max()
    
    #Calculates the number of students and adds it to the dictionary
    total_students = len(school['Student ID'])
    school_dict['Total Students'] = total_students
    
    #Finds the school budget and adds it to the dictionary
    school_budget = school['budget'].max()
    school_dict['Total School Budget'] = school_budget
    
    #Calculates the budget per student students and adds it to the dictionary
    school_dict['Budget Per Student'] = school_budget/total_students
    
    #Calculates the average reading and math scores adds them to the dictionary
    school_dict['Average Math Score'] = school['math_score'].mean()
    school_dict['Average Reading Score'] = school['reading_score'].mean()
    
    #Locates the series of students who passed math and reading, respectively
    passing_math = school.loc[(school['math_score'] >= 70)]
    passing_reading = school.loc[(school['reading_score'] >= 70)]
    
    #Calculates the percentage of students who passed math and reading,
    #respectively and adds those numbers to the dictionary.
    school_dict['Passing Math'] = len(passing_math)*100/total_students
    school_dict['Passing Reading'] = len(passing_reading)*100/total_students
    
    #Locates the series of students who passed overall
    overall_passing = (len(passing_math) + len(passing_reading))/2
    
    #Note, the sample solution seems to just average the math % and the reading %, which I 
    #don't really agree with considering that these are school averages and it's likely
    #that locating the number of kids that actually passed is different. When locating the
    #number of kids who averaged out to passing overall (i.e. loc.(math/2 + reading/2) >= 70), 
    #I ended up with around 89% which seems off, but maybe it's possible 
    #that the individual data would lend itself to that possibility.
    
    #Calculates the percentage of overall passers and adds it to the dictionary
    school_dict['Overall Passing Rate'] = overall_passing*100/total_students

    #Adds the dictionary to the school list that came before the for-loop.
    total_school_dict_list.append({school_name:school_dict})

#Takes the first line of total_school_dict_list and turns it into a dataframe
existing_df = pd.DataFrame.from_dict(total_school_dict_list[0], orient='index')

#Loops through the rest of the schools in total_school_dict_list and makes them
#dataframes, also merging them to the existing_df that came before.
for i in range(1, len(total_school_dict_list)):
    pd.DataFrame.from_dict(total_school_dict_list[i], orient='index')
    new_df = pd.DataFrame.from_dict(total_school_dict_list[i], orient='index')
    existing_df = pd.concat([existing_df, new_df])

#Renames our existing_df to a more appropriate name
school_df = existing_df

#Maps our df columns to a better format
school_df["Average Math Score"] = school_df["Average Math Score"].map("{:.2f}".format)
school_df["Average Reading Score"] = school_df["Average Reading Score"].map("{:.2f}".format)
school_df["Passing Math"] = school_df["Passing Math"].map("{:.2f}%".format)
school_df["Passing Reading"] = school_df["Passing Reading"].map("{:.2f}%".format)
school_df["Total School Budget"] = school_df["Total School Budget"].map("${:,.2f}".format)
school_df["Budget Per Student"] = school_df["Budget Per Student"].map("${:,.2f}".format)
school_df["Total Students"] = school_df["Total Students"].map("{:,}".format)
school_df["Overall Passing Rate"] = school_df["Overall Passing Rate"].map("{:.2f}%".format)


#Sorts our values based on those that had the best overall passing rate
top_performing_df = school_df.sort_values(
    ["Overall Passing Rate"], ascending=False)
top_performing_df.head()


,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,95.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,95.27%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,95.27%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,95.20%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [4]:
#Takes the same display as before, just changing 'ascending' from 'False' to 'True'
bottom_performing_df = school_df.sort_values(
    ["Overall Passing Rate"], ascending=True)
bottom_performing_df.head()

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,73.29%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,73.64%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,73.80%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [5]:
#Locates the students for each of the respective grades.
ninth_df = school_data_complete.loc[(school_data_complete["grade"] == '9th'),:]
tenth_df = school_data_complete.loc[(school_data_complete["grade"] == '10th'),:]
eleventh_df = school_data_complete.loc[(school_data_complete["grade"] == '11th'),:]
twelveth_df = school_data_complete.loc[(school_data_complete["grade"] == '12th'),:]

#1) Sets the index to 'school_name', 2) Groups the data by school_name
#3) Creates a dataframe of the mean score for each school.
#4) Renames the math_score column to the respective grade.
ninth_df = ninth_df.set_index(['school_name'])
ninth_df = ninth_df.groupby(['school_name'])
ninth_df = pd.DataFrame(ninth_df['math_score'].mean())
ninth_df = ninth_df.rename(columns={"math_score" : "9th Grade"})

#Repeats the process for 10th, 11th, and 12th grades.
tenth_df = tenth_df.set_index(['school_name'])
tenth_df = tenth_df.groupby(['school_name'])
tenth_df = pd.DataFrame(tenth_df['math_score'].mean())
tenth_df = tenth_df.rename(columns={"math_score" : "10th Grade"})

eleventh_df = eleventh_df.set_index(['school_name'])
eleventh_df = eleventh_df.groupby(['school_name'])
eleventh_df = pd.DataFrame(eleventh_df['math_score'].mean())
eleventh_df = eleventh_df.rename(columns={"math_score" : "11th Grade"})

twelveth_df = twelveth_df.set_index(['school_name'])
twelveth_df = twelveth_df.groupby(['school_name'])
twelveth_df = pd.DataFrame(twelveth_df['math_score'].mean())
twelveth_df = twelveth_df.rename(columns={"math_score" : "12th Grade"})

#Merges all the dataframes into one final dataframe.
merge_table_1 = pd.merge(ninth_df, tenth_df, on="school_name")
merge_table_2 = pd.merge(eleventh_df, twelveth_df, on="school_name")
merge_table_3 = pd.merge(merge_table_1, merge_table_2, on="school_name")
merge_table_3

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [6]:
#Locates the students for each of the respective grades.
ninth_df = school_data_complete.loc[(school_data_complete["grade"] == '9th'),:]
tenth_df = school_data_complete.loc[(school_data_complete["grade"] == '10th'),:]
eleventh_df = school_data_complete.loc[(school_data_complete["grade"] == '11th'),:]
twelveth_df = school_data_complete.loc[(school_data_complete["grade"] == '12th'),:]

#1) Sets the index to 'school_name', 2) Groups the data by school_name
#3) Creates a dataframe of the mean score for each school.
#4) Renames the math_score column to the respective grade.
ninth_df = ninth_df.set_index(['school_name'])
ninth_df = ninth_df.groupby(['school_name'])
ninth_df = pd.DataFrame(ninth_df['reading_score'].mean())
ninth_df = ninth_df.rename(columns={'reading_score' : "9th Grade"})

#Repeats the process for 10th, 11th, and 12th grades.
tenth_df = tenth_df.set_index(['school_name'])
tenth_df = tenth_df.groupby(['school_name'])
tenth_df = pd.DataFrame(tenth_df['reading_score'].mean())
tenth_df = tenth_df.rename(columns={'reading_score' : "10th Grade"})

eleventh_df = eleventh_df.set_index(['school_name'])
eleventh_df = eleventh_df.groupby(['school_name'])
eleventh_df = pd.DataFrame(eleventh_df['reading_score'].mean())
eleventh_df = eleventh_df.rename(columns={'reading_score' : "11th Grade"})

twelveth_df = twelveth_df.set_index(['school_name'])
twelveth_df = twelveth_df.groupby(['school_name'])
twelveth_df = pd.DataFrame(twelveth_df['reading_score'].mean())
twelveth_df = twelveth_df.rename(columns={'reading_score' : "12th Grade"})

#Merges all the dataframes into one final dataframe.
merge_table_1 = pd.merge(ninth_df, tenth_df, on="school_name")
merge_table_2 = pd.merge(eleventh_df, twelveth_df, on="school_name")
merge_table_3 = pd.merge(merge_table_1, merge_table_2, on="school_name")
merge_table_3

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [7]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [8]:
#Removes Currency Format of 'Budget Per Student' (Can only run once)
existing_df['Budget Per Student'] = existing_df['Budget Per Student'].str.replace('$', '')
existing_df['Total Students'] = existing_df['Total Students'].str.replace(',', '')
existing_df['Passing Math'] = existing_df['Passing Math'].str.replace('%', '')
existing_df['Passing Reading'] = existing_df['Passing Reading'].str.replace('%', '')

In [9]:
existing_df

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
Huang High School,District,2917,"$1,910,635.00",655.00,76.63,81.18,65.68,81.32,73.50%
Figueroa High School,District,2949,"$1,884,411.00",639.00,76.71,81.16,65.99,80.74,73.36%
Shelton High School,Charter,1761,"$1,056,600.00",600.00,83.36,83.73,93.87,95.85,94.86%
Hernandez High School,District,4635,"$3,022,020.00",652.00,77.29,80.93,66.75,80.86,73.81%
Griffin High School,Charter,1468,"$917,500.00",625.00,83.35,83.82,93.39,97.14,95.27%
Wilson High School,Charter,2283,"$1,319,574.00",578.00,83.27,83.99,93.87,96.54,95.20%
Cabrera High School,Charter,1858,"$1,081,356.00",582.00,83.06,83.98,94.13,97.04,95.59%
Bailey High School,District,4976,"$3,124,928.00",628.00,77.05,81.03,66.68,81.93,74.31%
Holden High School,Charter,427,"$248,087.00",581.00,83.80,83.81,92.51,96.25,94.38%
Pena High School,Charter,962,"$585,858.00",609.00,83.84,84.04,94.59,95.95,95.27%


In [10]:
#Converts Budget to a float so that we can bin it.
existing_df["Budget Per Student"] = existing_df['Budget Per Student'].astype(float)
existing_df["Average Math Score"] = existing_df['Average Math Score'].astype(float)
existing_df["Average Reading Score"] = existing_df['Average Reading Score'].astype(float)
existing_df["Passing Math"] = existing_df['Passing Math'].astype(float)
existing_df["Passing Reading"] = existing_df['Passing Reading'].astype(float)
existing_df["Total Students"] = existing_df['Total Students'].astype(int)

#Bins each school into one of these spending ranges
existing_df["Spending Ranges (Per Student)"] = pd.cut(existing_df["Budget Per Student"], 
                                                      spending_bins, labels=group_names)



In [11]:
def make_data_frame(columns_list, values_list):
#Puts our values and columns into different lists and makes them a dataframe so that I don't
#have to keep looking up the documentation. Also sets the index for us. (May be extraneous
#code but at this point I'm not sure I want to fix something that isn't broken.)

    df = pd.DataFrame(values_list, columns = columns_list)
    df.set_index([columns_list[0]])
    
    #Turns df_dict into a DataFrame
    return df

In [22]:
#Changes our dataframe to a more appropriate name
spending_df = existing_df

#Groups and sets the index for our spending bins.
spending_df.groupby('Spending Ranges (Per Student)')
a_df = spending_df.set_index('Spending Ranges (Per Student)')

a_df = a_df.sort_values(
    ["Spending Ranges (Per Student)"], ascending=True)
a_df


,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate,School Size
Spending Ranges (Per Student),,,,,,,,,,
<$585,Charter,2283,"$1,319,574.00",578.0,83.27,83.99,93.87,96.54,95.20%,Large (2000-5000)
<$585,Charter,1858,"$1,081,356.00",582.0,83.06,83.98,94.13,97.04,95.59%,Medium (1000-2000)
<$585,Charter,427,"$248,087.00",581.0,83.80,83.81,92.51,96.25,94.38%,Small (<1000)
<$585,Charter,1800,"$1,049,400.00",583.0,83.68,83.95,93.33,96.61,94.97%,Medium (1000-2000)
$585-615,Charter,1761,"$1,056,600.00",600.0,83.36,83.73,93.87,95.85,94.86%,Medium (1000-2000)
$585-615,Charter,962,"$585,858.00",609.0,83.84,84.04,94.59,95.95,95.27%,Small (<1000)
$615-645,District,2949,"$1,884,411.00",639.0,76.71,81.16,65.99,80.74,73.36%,Large (2000-5000)
$615-645,Charter,1468,"$917,500.00",625.0,83.35,83.82,93.39,97.14,95.27%,Medium (1000-2000)
$615-645,District,4976,"$3,124,928.00",628.0,77.05,81.03,66.68,81.93,74.31%,Large (2000-5000)


In [13]:
def bin_df_maker(binned_dataframe, index):

    #Creates empty counters for the bin's district totals that we will get summary
    #statistics from later.
    t_students = binned_dataframe['Total Students'].sum()
    av_math = 0
    av_read = 0
    av_math_rate = 0
    av_read_rate = 0
    
    #Loops through every school in the binned_dataframe to calculate summary statistics
    #and weighs each schools summary statistics based on the size of the school.
    for i in range(0, len(binned_dataframe['Total Students'])):
        
        #As you can see, I'm weighing the scores based on school size. That way, smaller
        #schools will not have an outsize influence on the overall average.
        
        #Finds our summary totals that we will later put into a dataframe.
        weighted_math_av = binned_dataframe['Total Students'][i]*binned_dataframe['Average Math Score'][i]
        weighted_read_av = binned_dataframe['Total Students'][i]*binned_dataframe['Average Reading Score'][i]
        weighted_math_rate_av = binned_dataframe['Total Students'][i]*binned_dataframe['Passing Math'][i]
        weighted_read_rate_av = binned_dataframe['Total Students'][i]*binned_dataframe['Passing Reading'][i]
        av_math = av_math + weighted_math_av
        av_read = av_read + weighted_read_av
        av_math_rate = av_math_rate + weighted_math_rate_av
        av_read_rate = av_read_rate + weighted_read_rate_av
    
    #Earlier, we multiplied each score by its school's size, now we need to divide our total
    #by the overall number of students in the district to remedy that earlier multiplication.
    av_math = av_math/t_students
    av_read = av_read/t_students
    av_math_rate = av_math_rate/t_students
    av_read_rate = av_read_rate/t_students
    av_overall = (av_math_rate + av_read_rate) / 2
    
    #Puts all of our columns and summary values into lists.
    list_val = [binned_dataframe[index].max(), 
                av_math, av_read, av_math_rate, av_read_rate, av_overall],
    col_val = [index, 'Average Math Score', 'Average Reading Score', 
              'Passing Math', 'Passing Reading', 'Overall Passing Rate']
    
    #Takes those lists and makes them a dataframe, later setting the dataframe to the 
    #desired index.
    bin_df = make_data_frame(col_val, list_val)
    bin_df = bin_df.set_index([index])
    
    #Returns our final dataframe that should be one row for the selected bin.
    return bin_df

In [14]:
#Fetches the dataframes of the schools in each bin
low_budget = spending_df.loc[spending_df["Spending Ranges (Per Student)"] == "<$585", :]
less_low_budget =spending_df.loc[spending_df["Spending Ranges (Per Student)"] == "$585-615", :]
less_high_budget =spending_df.loc[spending_df["Spending Ranges (Per Student)"] == "$615-645", :]
high_budget =spending_df.loc[spending_df["Spending Ranges (Per Student)"] == "$645-675", :]

#Uses the bin_df_maker() function to turn each of the previous dfs into summary rows
bin_1 = bin_df_maker(low_budget, "Spending Ranges (Per Student)")
bin_2 = bin_df_maker(less_low_budget, "Spending Ranges (Per Student)")
bin_3 = bin_df_maker(less_high_budget, "Spending Ranges (Per Student)")
bin_4 = bin_df_maker(high_budget, "Spending Ranges (Per Student)")

#Concatenates the rows into our final dataframe
summary_spending_df = pd.concat([bin_1, bin_2, bin_3, bin_4])
summary_spending_df

,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.360159,83.963706,93.702029,96.686226,95.194128
$585-615,83.529578,83.839519,94.124367,95.885329,95.004848
$615-645,78.060799,81.433195,71.401096,83.614308,77.507702
$645-675,77.048577,81.004693,66.229714,81.108175,73.668945


## Scores by School Size

* Perform the same operations as above, based on school size.

In [15]:
# Sample bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [16]:
#Cuts the data based on the bins provided and creates a new row.
existing_df["School Size"] = pd.cut(existing_df["Total Students"], 
                                                      size_bins, labels=group_names)

#Changes the name of the df to a more appropriate one and groups by the bin.
size_df = existing_df
size_df.groupby('School Size')
size_df

,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate,Spending Ranges (Per Student),School Size
Huang High School,District,2917,"$1,910,635.00",655.0,76.63,81.18,65.68,81.32,73.50%,$645-675,Large (2000-5000)
Figueroa High School,District,2949,"$1,884,411.00",639.0,76.71,81.16,65.99,80.74,73.36%,$615-645,Large (2000-5000)
Shelton High School,Charter,1761,"$1,056,600.00",600.0,83.36,83.73,93.87,95.85,94.86%,$585-615,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",652.0,77.29,80.93,66.75,80.86,73.81%,$645-675,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",625.0,83.35,83.82,93.39,97.14,95.27%,$615-645,Medium (1000-2000)
Wilson High School,Charter,2283,"$1,319,574.00",578.0,83.27,83.99,93.87,96.54,95.20%,<$585,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.06,83.98,94.13,97.04,95.59%,<$585,Medium (1000-2000)
Bailey High School,District,4976,"$3,124,928.00",628.0,77.05,81.03,66.68,81.93,74.31%,$615-645,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",581.0,83.80,83.81,92.51,96.25,94.38%,<$585,Small (<1000)
Pena High School,Charter,962,"$585,858.00",609.0,83.84,84.04,94.59,95.95,95.27%,$585-615,Small (<1000)


In [17]:
#Locates the data for each bin and makes them into a dataframe
small = size_df.loc[size_df["School Size"] == "Small (<1000)", :]
medium = size_df.loc[size_df["School Size"] == "Medium (1000-2000)", :]
large =size_df.loc[size_df["School Size"] == "Large (2000-5000)", :]


#Uses the bin_df_maker() function to turn each of the previous dfs into summary rows
bin_1 = bin_df_maker(small, "School Size")
bin_2 = bin_df_maker(medium, "School Size")
bin_3 = bin_df_maker(large, "School Size")


#Concatenates the rows into our final dataframe
summary_size_df = pd.concat([bin_1, bin_2, bin_3])
summary_size_df

,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.827703,83.969294,93.950576,96.042225,94.996400
Medium (1000-2000),83.371971,83.869500,93.614830,96.772300,95.193565
Large (2000-5000),77.476441,81.197566,68.652742,82.124276,75.388509


## Scores by School Type

* Perform the same operations as above, based on school type.

In [18]:
#Repeats the same operation as before, but for School Type.
district = size_df.loc[size_df["School Type"] == "District", :]
charter = size_df.loc[size_df["School Type"] == "Charter", :]


bin_1 = bin_df_maker(charter, "School Type")
bin_2 = bin_df_maker(district, "School Type")

summary_type_df = pd.concat([bin_1, bin_2])
summary_type_df                             

,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.404792,83.903428,93.700848,96.645646,95.173247
District,76.986128,80.961240,66.518586,80.904262,73.711424
